In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
import os

In [ ]:
!nvidia-smi -L

GPU 0: Tesla K80 (UUID: GPU-ab88920b-e850-f13c-e6f2-36b12f2f5fbb)


In [ ]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

#Get helper functions
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2022-05-09 17:59:17--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-05-09 17:59:18 (81.3 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [ ]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

#Unzip dataset
unzip_data('nlp_getting_started.zip')

--2022-05-09 17:59:18--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.197.128, 64.233.191.128, 173.194.192.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.197.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.005s  

2022-05-09 17:59:18 (107 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [ ]:
#For us to import data using pandas, well need to fit all the data on the ram, which fails when the data size is huge
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
#Shuffle dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [ ]:
#Lets visualize some random training samples
random_index= random.randint(0, len(train_df_shuffled)-5)
for row in train_df_shuffled[['text', 'target']][random_index:random_index+5].itertuples():
  _, text, target = row
  print (f"Target : {target}","(real disaster)" if target > 0 else "(not real disaster)")
  print (f"Text : \n{text}\n")
  print ("------\n")

Target : 1 (real disaster)
Text : 
Thunder pounds north goes black
a deep bruise on the sky's chest
wind cries its pain.  
A summer storm has a tough life
short violent.

------

Target : 0 (not real disaster)
Text : 
@emmerdale is Ross really dead?? #AskCharley

------

Target : 1 (real disaster)
Text : 
Photo: blue by @forest.fires source: http://t.co/awXR24zsqh http://t.co/o9A26Fn27y

------

Target : 0 (not real disaster)
Text : 
I added a video to a @YouTube playlist http://t.co/f2TqMFh1Yb Cher Lloyd - Sirens

------

Target : 0 (not real disaster)
Text : 
'60 all out? What!' - World reacts to Aussie collapse http://t.co/I6zQlk2Puz

------



In [ ]:
from sklearn.model_selection import train_test_split

train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled['text'].to_numpy(),
                                                                            train_df_shuffled['target'].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=42)

In [ ]:
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [ ]:
from tensorflow.keras.layers import TextVectorization

#Using default Parameters
text_vectorizer = TextVectorization(max_tokens=None,
                                    standardize='lower_and_strip_punctuation',
                                    split='whitespace',
                                    ngrams=None,
                                    output_mode='int',
                                    output_sequence_length=None,
                                    pad_to_max_tokens=False
                                    )

#Find the average number of tokens in the training tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [ ]:
#Now set up vectorization variables
max_vocab_length=10000
max_length=15

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode='int',
                                    output_sequence_length=max_length,
                                    )

In [ ]:
#Fit the vectorizer on the training data
text_vectorizer.adapt(train_sentences)

In [ ]:
SAVE_DIR = 'model_logs'

In [ ]:
###Creating an embedding layer using Tensorflows embedding layer
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim = max_vocab_length,
                             output_dim = 128,
                             input_length = max_length)
embedding

In [ ]:
#randomly choose a sample from train sentences and see
random_sentence = random.choice(train_sentences)
print (f'Original Sentence :\n {random_sentence}\
        \n\nEmbedded Version : \n')
embedding(text_vectorizer([random_sentence])) #because embedding layer takes integers as input and not words

Original Sentence :
 Cue storm with massive wind and hail!! Now a power outage @WXII in Davidson County.        

Embedded Version : 



<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.00603254,  0.04733125, -0.01328781, ...,  0.01372525,
         -0.00315313, -0.04480665],
        [ 0.01630649,  0.03633583,  0.01413565, ...,  0.00456144,
          0.00363611, -0.01776563],
        [ 0.02392597,  0.00032611, -0.04776685, ..., -0.01243216,
         -0.02829002,  0.04959175],
        ...,
        [-0.01185553,  0.04928113,  0.0436285 , ...,  0.00821568,
          0.02482957,  0.01728251],
        [-0.01663973, -0.02396926,  0.02076974, ..., -0.00710518,
         -0.02947006,  0.021279  ],
        [-0.01802886, -0.044267  , -0.00532248, ...,  0.01737818,
          0.00843646, -0.03490759]]], dtype=float32)>

###Now build Model 6 using Transfer Learning

In [ ]:
import tensorflow_hub as hub

sample_sentence = "Bishwas!!!, Theres a flood in my street!"

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
embed_samples = embed([sample_sentence, "I am a sentence for which I would like to get its embedding"])

In [ ]:
#Check if the embeddings are of length 512
embed_samples, embed_samples[0].shape

(<tf.Tensor: shape=(2, 512), dtype=float32, numpy=
 array([[ 0.02067665,  0.02442249,  0.02530091, ...,  0.00296551,
          0.01766311, -0.01987905],
        [ 0.05080862, -0.0165243 ,  0.01573781, ...,  0.00976658,
          0.0317012 ,  0.01788118]], dtype=float32)>, TensorShape([512]))

In [ ]:
#Create a keras layer using pretrained layer
sentence_encoder_layer = hub.KerasLayer('https://tfhub.dev/google/universal-sentence-encoder/4',
                                        input_shape=[],
                                        dtype=tf.string,
                                        trainable=False,
                                        name='USE_Pretrained')

In [ ]:
#Now create the model using a sequentila API
model_6 = tf.keras.Sequential([
                               sentence_encoder_layer,
                               layers.Dense(64, activation='relu'),
                               layers.Dense(1, activation='sigmoid')
], name='model_6_USE')

model_6.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

model_6.summary()

Model: "model_6_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE_Pretrained (KerasLayer)  (None, 512)              256797824 
                                                                 
 dense (Dense)               (None, 64)                32832     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [ ]:
model_6_history = model_6.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data = (val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     'model_6_Pretrained')])

Saving TensorBoard log files to: model_logs/model_6_Pretrained/20220509-175946
Epoch 1/5
215/215 [==============================] - 8s 21ms/step - loss: 0.5058 - accuracy: 0.7860 - val_loss: 0.4485 - val_accuracy: 0.8018
Epoch 2/5
215/215 [==============================] - 4s 18ms/step - loss: 0.4140 - accuracy: 0.8161 - val_loss: 0.4389 - val_accuracy: 0.8136
Epoch 3/5
215/215 [==============================] - 4s 17ms/step - loss: 0.4013 - accuracy: 0.8221 - val_loss: 0.4304 - val_accuracy: 0.8163
Epoch 4/5
215/215 [==============================] - 4s 19ms/step - loss: 0.3920 - accuracy: 0.8262 - val_loss: 0.4268 - val_accuracy: 0.8176
Epoch 5/5
215/215 [==============================] - 4s 17ms/step - loss: 0.3861 - accuracy: 0.8259 - val_loss: 0.4262 - val_accuracy: 0.8176


In [ ]:
#Creating a function to calculate all the evaluation metrics

from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  model_accuracy = accuracy_score(y_true, y_pred)*100
  model_precision, model_recall, f1_score, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')

  model_results = {"accuracy" : model_accuracy,
                   "precision" : model_precision,
                   "recall" : model_recall,
                   "f1_score" : f1_score}

  return model_results

In [ ]:
model_6_pred_probs = model_6.predict(val_sentences)
model_6_pred_probs = tf.squeeze(tf.round(model_6_pred_probs))
model_6_results = calculate_results(y_true=val_labels,
                                    y_pred=model_6_pred_probs)
model_6_results

{'accuracy': 81.75853018372703,
 'f1_score': 0.8161793631328899,
 'precision': 0.8196403099324437,
 'recall': 0.8175853018372703}

###Model 7 - Prtrained USE but with 10% of the total data

In [ ]:
#Create subsets of 10% of the training data

#This causes data leakage problem. Coz a few samples from the validation set are already present in training split and causes your model to look like its doung better
#train_10_percent = train_df_shuffled[['text','target']].sample(frac=0.1, random_state=42)
#train_sentences_10_percent = train_10_percent['text'].to_list()
#train_labels_10_percent = train_10_percent['target'].to_list()

#SOOO This is better
train_10_percent_split = int(0.1 * len(train_sentences))
train_sentences_10_percent = train_sentences[:train_10_percent_split]
train_labels_10_percent = train_labels[:train_10_percent_split]

In [ ]:
#Check the number of targets in our subset of data
#train_10_percent['target'].value_counts()

In [ ]:
#Now create the model using a sequentila API
model_7 = tf.keras.Sequential([
                               sentence_encoder_layer,
                               layers.Dense(64, activation='relu'),
                               layers.Dense(1, activation='sigmoid')
], name='model_7_USE')

model_7.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

model_7.summary()

Model: "model_7_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE_Pretrained (KerasLayer)  (None, 512)              256797824 
                                                                 
 dense_2 (Dense)             (None, 64)                32832     
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [ ]:
model_7_history = model_7.fit(train_sentences_10_percent,
                              train_labels_10_percent,
                              epochs=5,
                              validation_data = (val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     'model_7_Pretrained')])

Saving TensorBoard log files to: model_logs/model_7_Pretrained/20220509-180030
Epoch 1/5
22/22 [==============================] - 3s 70ms/step - loss: 0.6712 - accuracy: 0.6788 - val_loss: 0.6494 - val_accuracy: 0.7362
Epoch 2/5
22/22 [==============================] - 1s 46ms/step - loss: 0.6000 - accuracy: 0.8073 - val_loss: 0.5908 - val_accuracy: 0.7638
Epoch 3/5
22/22 [==============================] - 1s 46ms/step - loss: 0.5251 - accuracy: 0.8117 - val_loss: 0.5348 - val_accuracy: 0.7664
Epoch 4/5
22/22 [==============================] - 1s 47ms/step - loss: 0.4623 - accuracy: 0.8175 - val_loss: 0.5045 - val_accuracy: 0.7730
Epoch 5/5
22/22 [==============================] - 1s 32ms/step - loss: 0.4224 - accuracy: 0.8277 - val_loss: 0.4904 - val_accuracy: 0.7730


In [ ]:
model_7_pred_probs = model_7.predict(val_sentences)
model_7_pred_probs = tf.squeeze(tf.round(model_7_pred_probs))
model_7_results = calculate_results(y_true=val_labels,
                                    y_pred=model_7_pred_probs)
model_7_results

{'accuracy': 77.29658792650919,
 'f1_score': 0.7716665369372698,
 'precision': 0.7733211200948874,
 'recall': 0.7729658792650919}

###Upload the logs to tensorboard now

In [ ]:
!tensorboard dev upload --logdir ./model_logs/ \
  --name 'NLP modelling experiments for Model 6 and Model 7' \
  --description 'Comparing the results of the two transfer learning models built' \
  --one_shot #exit the uploader once uploading is finished


***** TensorBoard Uploader *****

This will upload your TensorBoard logs to https://tensorboard.dev/ from
the following directory:

./model_logs/

This TensorBoard will be visible to everyone. Do not upload sensitive
data.

Your use of this service is subject to Google's Terms of Service
<https://policies.google.com/terms> and Privacy Policy
<https://policies.google.com/privacy>, and TensorBoard.dev's Terms of Service
<https://tensorboard.dev/policy/terms/>.

This notice will not be shown again while you are logged into the uploader.
To log out, run `tensorboard dev auth revoke`.

Continue? (yes/NO) yes

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=373649185512-8v619h5kft38l4456nm2dj4ubeqsrvh6.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email&state=xL6vFdBbQxeCXzBQQSSm9e4vkeSiLC&prompt=consent&access_type=offline

In [ ]:
#If we need to delete an experiment from Tensorboard, we need to run this

#To pull out the list of experiments that have run
#!tensorboard dev list

#Now delete it using the ID
#!tensorboard dev delete --experiment_id 2eLdxFnHRXSBUYxmDoOpLw

https://tensorboard.dev/experiment/2eLdxFnHRXSBUYxmDoOpLw/
	Name                 NLP modelling experiments for Model 6 and Model 7
	Description          Comparing the results of the two transfer learning models built
	Id                   2eLdxFnHRXSBUYxmDoOpLw
	Created              2022-05-09 18:24:17 (6 minutes ago)
	Updated              2022-05-09 18:24:19 (6 minutes ago)
	Runs                 4
	Tags                 5
	Scalars              60
	Tensor bytes         0
	Binary object bytes  1862752
Total: 1 experiment(s)
Deleted experiment 2eLdxFnHRXSBUYxmDoOpLw.


###Saving and loading the NLP models to execute it again 

In [ ]:
model_6_results

{'accuracy': 81.75853018372703,
 'f1_score': 0.8161793631328899,
 'precision': 0.8196403099324437,
 'recall': 0.8175853018372703}

In [ ]:
#Save model 6 to HDF5 format
model_6.save('model_6.h5')

In [ ]:
#Now load it back in with custom Hub layer
loaded_model_6 = tf.keras.models.load_model('model_6.h5',
                                     custom_objects={'KerasLayer': hub.KerasLayer}) #Required because were using a transfer learning hub layer

loaded_model_6.summary()

Model: "model_6_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE_Pretrained (KerasLayer)  (None, 512)              256797824 
                                                                 
 dense (Dense)               (None, 64)                32832     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [ ]:
#Now lets save to the SaveModel format
model_6.save('model_6_SaveModel_format')

INFO:tensorflow:Assets written to: model_6_SaveModel_format/assets


INFO:tensorflow:Assets written to: model_6_SaveModel_format/assets


In [ ]:
loaded_model_6_savedmodel_format = tf.keras.models.load_model('model_6_SaveModel_format')
loaded_model_6_savedmodel_format.summary()

Model: "model_6_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE_Pretrained (KerasLayer)  (None, 512)              256797824 
                                                                 
 dense (Dense)               (None, 64)                32832     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________
